In [ ]:
from itertools import islice
import os
import sys
import numpy as np # paper : 10.1038/s41586-020-2649-2
import pandas as pd # 10.5281/zenodo.13819579
import shutil
import chemcoord as cc  # paper: 10.1002/jcc.27029 ; documentation : http://chemcoord.readthedocs.org/

In [ ]:
def split_file_by_lines(filename, lines_per_subfile):
    with open(os.getcwd()+'/crest_conformers.xyz', 'r') as file:
        conf_num = 1
        while True:
            # Read the next chunk of lines
            conf = list(islice(file, lines_per_subfile))
            if not conf:
                break  # Stop if there are no more lines to read
            
            # Process the chunk (e.g., save to another file or process further)
            with open(os.getcwd()+'/conformers/conformer_'+str(conf_num)+'.xyz', 'w') as conf_file:
                conf_file.writelines(conf)
                
            conf_num += 1
                
    print(f"{subfiles_number} subfiles were generated with an extension of {lines_per_subfile} lines per file.")                

In [ ]:
input_file = open(os.getcwd()+'/crest_conformers.xyz', 'rb')

os.mkdir(os.getcwd()+'/conformers')

first_line = input_file.readline() # the number of atoms in the system is retrieved from the first line in the xyz file

with input_file as f:              # _with_ function is used to count the length of the xyz file. https://stackoverflow.com/a/1019572
    num_lines = sum(1 for _ in f)
    num_lines = num_lines+1

lines_per_subfile = int(first_line) + 2
subfiles_number = num_lines/lines_per_file

split_file_by_lines('crest_conformers.xyz', lines_per_subfile) # calling the defined function, 
                                                               # we will get the subfiles with 1 conformer each

In [ ]:
## Calculate_rmsd program is required to be used through bash terminal. It calculates the rmsd in an array of strcutures.

%%bash
cd conformers
    
for a in {1..362}
do
    for b in {1..362}
    do
    c=$(($b+$a))
    d=$((362))
    if [[ $c -le $d ]]; 
        then (echo $a
        echo $c
        calculate_rmsd --reorder ./conformer_$a.xyz ./conformer_$c.xyz ) >> raw_rmsd.csv
    fi
    done
done
    

In [ ]:
## First refinement step based on rmsd tolerances (1 A) between the structures.

path = os.getcwd()+'/conformers/raw_rmsd.csv'
rmsd_csv = pd.read_csv(path, header=None)

lenght = len(rmsd_csv)
total = range(0, lenght, 3)
total_list = list(total)

redundant_i = []

for i in total_list:
    
    if rmsd_csv.values[i+2][0] <= 1:
        redundant_i.append(rmsd_csv.values[i+1][0])
        
    else:
       pass

redundant_clean = set(redundant_i)
print(f'Total number of unique structures in the first refinement cycle: {362-len(redundant_clean)}')

In [ ]:
clean_conf = []

X = []
Y = []
Z = []

for i in total_list:
    new_i = rmsd_csv.values[i][0]
    
    if new_i not in redundant_clean:
        new_i_2 = rmsd_csv.values[i+1][0]
        
        if new_i_2 not in redundant_clean:
            X_temp = rmsd_csv.values[i][0]
            Y_temp = rmsd_csv.values[i+1][0]
            Z_temp = rmsd_csv.values[i+2][0]
            
            X.append(X_temp)
            Y.append(Y_temp)
            Z.append(Z_temp)
    
            clean_conf.append(rmsd_csv.values[i][0])
    else:
        pass
        
clean_conf_set = set(clean_conf)
clean_conf_list = sorted(list(clean_conf_set))
print(f'Remaining unique structures are listed: \n \n {clean_conf_list}')

In [ ]:
new_conf_number = len(clean_conf_list)

molecule = []
for i in range(0,new_conf_number):
    new_i = clean_conf_list[i]
    print(f'Obtaining data from structure {new_i}...')   
    xyz_file = os.getcwd()+'/conformers/conformer_'+str(int(new_i))+'.xyz'
    molecule_new_i = cc.Cartesian.read_xyz(xyz_file)
   
    print('Cartesian coordinates were succesfully retrieved...')
    print(' ')
    molecule.append(molecule_new_i)

print('#### Process was completed ####')

In [ ]:
## Second refinement step based on Cu-C distances of 5 A

new_conf_number = len(clean_conf_list)
clean_indexes = []

for i in range(0,new_conf_number):
    Cu = molecule[i].loc[0, ['x','y','z']]  # First number represents the atom index position in the XYZ file 
    C = molecule[i].loc[26, ['x','y','z']]  # while [x,y,z] represents the cartesian coordinates of the atom
    d = Cu - C                              # Vector between both atoms is calculated
    mod_d = np.linalg.norm(d)               # Module of the vector is calculated to obtain the distance

    if mod_d < 5:                           # Arbitrary threshold fot filtering structures with Cu-C distances > 5 A
        clean_indexes.append(i)
    else:
        pass

## Obtention of the corresponding conformer number and length of resultig list after second refinement

n_clean_conf_list = []                      

for i in clean_indexes: 
    new_i = int(i)
    n_clean_conf_list.append(clean_conf_list[new_i])
    
print(n_clean_conf_list) 
print(len(n_clean_conf_list))

In [ ]:
## Third refinement step based on C1-C1 distances of 6.5 A

n_clean_indexes = []

for i in clean_indexes:
    new_i = int(i)
    C1 = molecule[new_i].loc[47, ['x','y','z']]
    C2 = molecule[new_i].loc[42, ['x','y','z']]
    d = C1 - C2
    mod_d = np.linalg.norm(d)
    
    if mod_d > 6.5:
        n_clean_indexes.append(i)
    else:
        pass

## Obtention of the corresponding conformer number and length of resultig list after second refinement

nn_clean_conf_list = []

for i in n_clean_indexes: 
    new_i = int(i)
    nn_clean_conf_list.append(clean_conf_list[new_i])
print(nn_clean_conf_list) 
print(len(nn_clean_conf_list))

In [ ]:
inal_conf_list = len(nn_clean_conf_list)
os.mkadir(os.getcwd()+'/conformers/final_conformers')


for i in range(0,final_conf_list):
    new_i = nn_clean_conf_list[i]
    print(f'Obtaining data from structure {new_i}...')   
    old_path = os.getcwd()+'/conformers/conformer_'+str(int(new_i))+'.xyz'
    new_path = os.getcwd()+'/conformers/final_conformers/conformer_'+str(int(new_i))+'.xyz'
    shutil.copyfile(old_path, new_path)
    
    print('Cartesian coordinates were succesfully retrieved...')
    print(' ')

print('###############################')
print('#### Process was completed ####')
print('###############################')